## Windowed aggregations using spark streaming and ingestion to the online feature store

### **Note: this notebook needs to be run with a PySpark kernel to work properly!*

In this notebook, we'll use the spark structured streaming for real time feature aggregations and writing them to the online and offline FS:

- Read data from the kafka topic .
- Time window aggregations using spark structured streaming.
- write aggregated feature to the online feature store


## Import necessary libraries

In [1]:
import json

from pyspark.sql.functions import from_json, window, avg,count, stddev, explode, date_format,col, mean
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, TimestampType, LongType, IntegerType

from hops import kafka, tls, hdfs

Starting Spark application


ID,Application ID,Kind,State,Spark UI,Driver log
8,application_1653093964210_0014,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
# name of the kafka topic to read card transactions from
KAFKA_TOPIC_NAME = "credit_card_transactions"

## Create a stream from the kafka topic


In [3]:
df_read = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka.get_broker_endpoints()) \
  .option("kafka.security.protocol",kafka.get_security_protocol()) \
  .option("kafka.ssl.truststore.location", tls.get_trust_store()) \
  .option("kafka.ssl.truststore.password", tls.get_key_store_pwd()) \
  .option("kafka.ssl.keystore.location", tls.get_key_store()) \
  .option("kafka.ssl.keystore.password", tls.get_key_store_pwd()) \
  .option("kafka.ssl.key.password", tls.get_trust_store_pwd()) \
  .option("kafka.ssl.endpoint.identification.algorithm", "") \
  .option("startingOffsets", "earliest")\
  .option("subscribe", KAFKA_TOPIC_NAME) \
  .load()

In [4]:
# Define schema to read from kafka topic 
parse_schema = StructType([StructField('tid', StringType(), True),
                           StructField('datetime', StringType(), True),
                           StructField('cc_num', StringType(), True),
                           StructField('category', StringType(), True),
                           StructField('amount', StringType(), True),
                           StructField('latitude', StringType(), True),
                           StructField('longitude', StringType(), True),
                           StructField('city', StringType(), True),
                           StructField('country', StringType(), True)]
                         )

In [5]:
# Deserialise data from and create streaming query
df_deser = df_read.selectExpr("CAST(value AS STRING)")\
                   .select(from_json("value", parse_schema).alias("value"))\
                   .select("value.tid", 
                           "value.datetime", 
                           "value.cc_num", 
                           "value.category", 
                           "value.amount", 
                           "value.latitude", 
                           "value.longitude",
                           "value.city",
                           "value.country")\
                   .selectExpr("CAST(tid as string)", 
                               "CAST(datetime as string)", 
                               "CAST(cc_num as long)", 
                               "CAST(category as string)", 
                               "CAST(amount as double)",
                               "CAST(latitude as double)",
                               "CAST(longitude as double)",
                               "CAST(city as string)",
                               "CAST(country as string)")

In [6]:
df_deser.isStreaming

True

In [7]:
df_deser.printSchema() 

root
 |-- tid: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- category: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)

In [18]:
def haversine(long, lat):
    """Compute Haversine distance between each consecutive coordinate in (long, lat)."""    
    from math import radians
    long = radians(long)
    lat = radians(lat)
    long_shifted = long.shift()
    lat_shifted = lat.shift()
    long_diff = long_shifted - long
    lat_diff = lat_shifted - lat

    a = np.sin(lat_diff/2.0)**2
    b = np.cos(lat) * np.cos(lat_shifted) * np.sin(long_diff/2.0)**2
    c = 2*np.arcsin(np.sqrt(a + b))

    return c

spark.udf.register("haversine", haversine)

<function haversine at 0x7fedcf8aef70>

## Create windowing aggregations over different time windows using spark streaming.

In [19]:
# 4 hour window
windowed4hTransactionDF =df_deser \
    .selectExpr("CAST(tid as string)", 
                "CAST(datetime as timestamp)", 
                "CAST(cc_num as long)", 
                "CAST(category as string)", 
                "CAST(amount as double)",
                "haversine(longitude, latitude) as loc_delta",
                "CAST(city as string)",
                "CAST(country as string)")\
    .withWatermark("datetime", "10 minutes") \
    .groupBy(window("datetime", "240 minutes"), "cc_num") \
    .agg(mean("amount").alias("trans_volume_mavg"), 
         stddev("amount").alias("trans_volume_mstd"), 
         count("cc_num").alias("trans_freq"),
         mean("loc_delta").alias("loc_delta_mavg")
        )\
    .select("cc_num", "trans_volume_mavg", "trans_volume_mstd", "trans_freq", "loc_delta_mavg")


In [20]:
windowed4hTransactionDF.isStreaming

True

In [21]:
windowed4hTransactionDF.printSchema()

root
 |-- cc_num: long (nullable = true)
 |-- trans_volume_mavg: double (nullable = true)
 |-- trans_volume_mstd: double (nullable = true)
 |-- trans_freq: long (nullable = false)
 |-- loc_delta_mavg: double (nullable = true)

### Establish a connection with your Hopsworks feature store.

In [22]:
import hsfs
connection = hsfs.connection()
# get a reference to the feature store, you can access also shared feature stores by providing the feature store name
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

## Get feature groups from hopsworks feature store.

In [23]:
transactions_4h_aggs = fs.get_feature_group("transactions_4h_aggs", version = 1)

## Insert streaming dataframes to the online feature group

Now we are ready to write this streaming dataframe as a long living application to the online storage of the other feature group.

In [ ]:
query_4h = transactions_4h_aggs.insert_stream(windowed4hTransactionDF)

### Check if spark streaming query is active

In [ ]:
query_4h.isActive

#### We can also check status of a query and if there are any exceptions trown.

In [ ]:
query_4h.status

In [ ]:
query_4h.exception()

### Lets check if data was ingested in to the online feature store

In [33]:
fs.sql("SELECT * FROM transactions_4h_aggs_agg_1",online=True).show(20,False)

+----------------+-----------------+------------------+------------------+
|cc_num          |num_trans_per_12h|avg_amt_per_12h   |stdev_amt_per_12h |
+----------------+-----------------+------------------+------------------+
|4444037300542691|7                |154.51857142857145|238.93552871214524|
|4609072304828342|7                |176.02714285714288|263.06316920176454|
|4161715127983823|10               |928.3030000000001 |1809.7934375689888|
|4223253728365626|13               |1201.686153846154 |2724.0564739389993|
|4572259224622748|9                |1291.5500000000002|2495.189283160699 |
|4436298663019939|11               |149.78636363636366|235.75729924109365|
|4159210768503456|6                |37.303333333333335|26.403001092047596|
|4231082502226286|10               |977.8430000000001 |2071.1095165208753|
|4090612125343330|15               |646.7259999999999 |1336.9214811370616|
|4416410688550228|11               |663.0627272727273 |1631.6188600717442|
|4853206196105715|10     

### Stop queries
If you are running this from a notebook, you can kill the Spark Structured Streaming Query by stopping the Kernel or by calling its `.stop()` method.

In [43]:
transactions_4h_aggs.stop()

## TODO (Davit): gif how to run offline backfill job